<a href="https://colab.research.google.com/github/Eduardo6446/BIMMO_IA/blob/main/BIMMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Este script es la versión 2 del prototipo del algoritmo de IA, ahora con la
capacidad de usar perfiles de mantenimiento genéricos si una moto específica
no se encuentra en la base de conocimiento.

Autor: Eduardo Orozco
Fecha: 20/10/2025
"""

# PASO 1: Simular nuestra base de conocimiento, AHORA INCLUYENDO PERFILES GENÉRICOS
datos_motos = {
    # --- MODELOS ESPECÍFICOS ---
    "Bajaj_Pulsar_NS200": {
      "info_moto": { "marca": "Bajaj", "modelo": "Pulsar NS200" },
      "tareas_mantenimiento": [
        { "componente_id": "aceite_motor", "componente_nombre_comun": "Aceite del Motor y Tamiz", "accion": "REEMPLAZAR", "intervalo": { "kilometros": 500 } },
        { "componente_id": "aceite_motor", "componente_nombre_comun": "Aceite del Motor", "accion": "REEMPLAZAR", "intervalo": { "kilometros": 5000 } },
        { "componente_id": "filtro_aceite", "componente_nombre_comun": "Filtro de Aceite (Papel)", "accion": "REEMPLAZAR", "intervalo": { "kilometros": 10000 } },
        { "componente_id": "filtro_aire", "componente_nombre_comun": "Filtro de Aire", "accion": "REEMPLAZAR", "intervalo": { "kilometros": 15000 } },
        { "componente_id": "bujias", "componente_nombre_comun": "Bujías (3)", "accion": "REEMPLAZAR", "intervalo": { "kilometros": 10000 } },
        { "componente_id": "kit_arrastre", "componente_nombre_comun": "Cadena de Transmisión", "accion": "LUBRICAR", "intervalo": { "kilometros": 500 } },
      ]
    },
    # --- PERFILES GENÉRICOS (NUEVO) ---
    "Generica_Trabajo_150cc": {
      "info_moto": { "marca": "Genérica", "modelo": "Moto de Trabajo 125-150cc (Carburada)" },
      "tareas_mantenimiento": [
        { "componente_id": "aceite_motor", "componente_nombre_comun": "Aceite del Motor", "accion": "REEMPLAZAR", "intervalo": { "kilometros": 2500 }, "notas_tecnicas": "Recomendación general. Consulta tu manual." },
        { "componente_id": "filtro_aire", "componente_nombre_comun": "Filtro de Aire (Espuma)", "accion": "LIMPIAR", "intervalo": { "kilometros": 2500 } },
        { "componente_id": "bujia", "componente_nombre_comun": "Bujía", "accion": "REEMPLAZAR", "intervalo": { "kilometros": 8000 } },
        { "componente_id": "kit_arrastre", "componente_nombre_comun": "Cadena de Transmisión", "accion": "LUBRICAR", "intervalo": { "kilometros": 500 } },
        { "componente_id": "frenos", "componente_nombre_comun": "Frenos (Tambor/Disco)", "accion": "INSPECCIONAR", "intervalo": { "kilometros": 2500 } }
      ]
    },
    "Generica_Urbana_250cc": {
      "info_moto": { "marca": "Genérica", "modelo": "Moto Urbana 160-250cc (Inyectada)" },
      "tareas_mantenimiento": [
        { "componente_id": "aceite_motor", "componente_nombre_comun": "Aceite del Motor", "accion": "REEMPLAZAR", "intervalo": { "kilometros": 4000 }, "notas_tecnicas": "Recomendación general. Consulta tu manual." },
        { "componente_id": "filtro_aceite", "componente_nombre_comun": "Filtro de Aceite", "accion": "REEMPLAZAR", "intervalo": { "kilometros": 8000 } },
        { "componente_id": "filtro_aire", "componente_nombre_comun": "Filtro de Aire (Papel)", "accion": "REEMPLAZAR", "intervalo": { "kilometros": 12000 } },
        { "componente_id": "bujia", "componente_nombre_comun": "Bujía (Iridio/Cobre)", "accion": "REEMPLAZAR", "intervalo": { "kilometros": 12000 } },
      ]
    }
}

# PASO 2: función para seleccionar el perfil de mantenimiento
def seleccionar_perfil_moto(modelo_buscado, cilindrada):
    """
    Busca un modelo específico y, si no lo encuentra, retorna un perfil genérico
    basado en la cilindrada.

    Args:
        modelo_buscado (str): El modelo a buscar (ej. "Bajaj_Pulsar_NS200").
        cilindrada (int): La cilindrada del motor en cc.

    Returns:
        str: La clave del perfil a utilizar en la base de conocimiento.
    """
    if modelo_buscado in datos_motos:
        print(f"-> Perfil encontrado: {modelo_buscado}")
        return modelo_buscado
    else:
        print(f"-> Modelo '{modelo_buscado}' no encontrado. Seleccionando perfil genérico.")
        if cilindrada >= 160:
            print("-> Usando perfil: Genérica Urbana 160-250cc")
            return "Generica_Urbana_250cc"
        else:
            print("-> Usando perfil: Genérica de Trabajo 125-150cc")
            return "Generica_Trabajo_150cc"

# PASO 3: El algoritmo central (modificado para usar el perfil seleccionado)
def calcular_probabilidad_mantenimiento(perfil_moto, kilometraje_actual):
    """
    Calcula la probabilidad o "Índice de Urgencia" de mantenimiento para
    el perfil de moto seleccionado.

    Args:
        perfil_moto (str): La clave del perfil de la moto (ej. "Bajaj_Pulsar_NS200").
        kilometraje_actual (int): El kilometraje actual de la moto en KM.

    Returns:
        list: Una lista de diccionarios con las recomendaciones ordenadas por urgencia.
    """
    tareas = datos_motos[perfil_moto]["tareas_mantenimiento"]
    resultados = []

    for tarea in tareas:
        intervalo_km = tarea["intervalo"].get("kilometros")
        if intervalo_km is not None and intervalo_km > 0:
            if kilometraje_actual % intervalo_km == 0 and kilometraje_actual > 0:
                probabilidad = 1.0
            else:
                km_en_ciclo_actual = kilometraje_actual % intervalo_km
                probabilidad = km_en_ciclo_actual / intervalo_km

            resultados.append({
                "componente": tarea["componente_nombre_comun"],
                "accion": tarea["accion"],
                "probabilidad": round(probabilidad, 2)
            })

    return sorted(resultados, key=lambda x: x["probabilidad"], reverse=True)

# PASO 4: Ejemplos de uso actualizados
if __name__ == "__main__":

    # --- EJEMPLO 1: Moto conocida ---
    print("--- CASO 1: MOTO CONOCIDA EN LA BASE DE DATOS ---")
    modelo_seleccionado_1 = "Bajaj_Pulsar_NS200"
    cilindrada_1 = 200
    kilometraje_usuario_1 = 9850

    perfil_1 = seleccionar_perfil_moto(modelo_seleccionado_1, cilindrada_1)

    print(f"\nAnálisis para: {datos_motos[perfil_1]['info_moto']['modelo']}")
    print(f"Kilometraje actual: {kilometraje_usuario_1} km\n")

    recomendaciones_1 = calcular_probabilidad_mantenimiento(perfil_1, kilometraje_usuario_1)

    print("Índice de Urgencia de Mantenimiento:")
    for reco in recomendaciones_1:
        barra = '█' * int(reco["probabilidad"] * 20)
        print(f"{reco['componente']:<30} {reco['accion']:<12} {reco['probabilidad']:.2f} |{barra}")

    print("\n" + "="*50 + "\n")

    # --- EJEMPLO 2: Moto desconocida (Fallback a genérico) ---
    print("--- CASO 2: MOTO DESCONOCIDA ---")
    modelo_seleccionado_2 = "Suzuki_Gixxer_150" # Este modelo no está en nuestra DB
    cilindrada_2 = 155
    kilometraje_usuario_2 = 7800

    perfil_2 = seleccionar_perfil_moto(modelo_seleccionado_2, cilindrada_2)

    print(f"\nAnálisis para: {modelo_seleccionado_2} (usando perfil: {datos_motos[perfil_2]['info_moto']['modelo']})")
    print(f"Kilometraje actual: {kilometraje_usuario_2} km\n")

    recomendaciones_2 = calcular_probabilidad_mantenimiento(perfil_2, kilometraje_usuario_2)

    print("Índice de Urgencia de Mantenimiento:")
    for reco in recomendaciones_2:
        barra = '█' * int(reco["probabilidad"] * 20)
        print(f"{reco['componente']:<30} {reco['accion']:<12} {reco['probabilidad']:.2f} |{barra}")

--- CASO 1: MOTO CONOCIDA EN LA BASE DE DATOS ---
-> Perfil encontrado: Bajaj_Pulsar_NS200

Análisis para: Pulsar NS200
Kilometraje actual: 9850 km

Índice de Urgencia de Mantenimiento:
Filtro de Aceite (Papel)       REEMPLAZAR   0.98 |███████████████████
Bujías (3)                     REEMPLAZAR   0.98 |███████████████████
Aceite del Motor               REEMPLAZAR   0.97 |███████████████████
Aceite del Motor y Tamiz       REEMPLAZAR   0.70 |██████████████
Cadena de Transmisión          LUBRICAR     0.70 |██████████████
Filtro de Aire                 REEMPLAZAR   0.66 |█████████████


--- CASO 2: MOTO DESCONOCIDA ---
-> Modelo 'Suzuki_Gixxer_150' no encontrado. Seleccionando perfil genérico.
-> Usando perfil: Genérica de Trabajo 125-150cc

Análisis para: Suzuki_Gixxer_150 (usando perfil: Moto de Trabajo 125-150cc (Carburada))
Kilometraje actual: 7800 km

Índice de Urgencia de Mantenimiento:
Bujía                          REEMPLAZAR   0.97 |███████████████████
Cadena de Transmisión     